In [3]:
import pandas as pd
import boto3
import sagemaker

In [2]:
# 1. Create Model Package Group
sm_client = boto3.client('sagemaker')

try:
    response = sm_client.create_model_package_group(
        ModelPackageGroupName='flight-delay-models',
        ModelPackageGroupDescription='Flight delay prediction models for airline operations'
    )
    print(f"✅ Model Package Group created: flight-delay-models")
except sm_client.exceptions.ResourceInUse:
    print("ℹ️  Model Package Group already exists: flight-delay-models")

# 2. Verify it exists
groups = sm_client.list_model_package_groups()
for group in groups['ModelPackageGroupSummaryList']:
    if group['ModelPackageGroupName'] == 'flight-delay-models':
        print(f"✅ Confirmed: {group['ModelPackageGroupName']} exists")
        print(f"   ARN: {group['ModelPackageGroupArn']}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 sm_client = boto3.client('sagemaker')                                                       │
│    6                                                                                             │
│    7 try:                                                                                        │
│ ❱  8 │   response = sm_client.create_model_package_group(                                        │
│    9 │   │   ModelPackageGroupName='flight-delay-models',                                        │
│   10 │   │   ModelPackageGroupDescription='Flight delay prediction models for airline operati    │
│   11 │   )                                                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ClientError: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model 
Package Group already exists: arn:aws:sagemaker:us-east-1:425709451100:model-package-group/flight-delay-models

In [6]:
"""
Generate Training CSVs from Parquet Files
Keeps parquet as-is (with metadata), creates CSVs with only numeric features
"""
import pandas as pd
import boto3

# Configuration
bucket = 'sagemaker-us-east-1-425709451100'
prefix = 'aai540-group1'

print("="*70)
print("GENERATING TRAINING CSVs FROM PARQUET")
print("="*70)

# ===========================
# DEFINE FEATURES FOR MODELING
# ===========================
model_features = [
    # Temporal features
    'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEP_HOUR', 'SCHEDULED_DEPARTURE',
    'HOUR_SIN', 'HOUR_COS', 'IS_PEAK_HOUR', 'IS_WEEKEND',
    # Distance features
    'DISTANCE', 'SCHEDULED_TIME', 'IS_LONG_HAUL', 'DISTANCE_BUCKET',
    # Target-encoded features
    'AIRLINE_DELAY_RATE', 'ORIGIN_DELAY_RATE', 'DEST_DELAY_RATE', 'ROUTE_DELAY_RATE',
    # Volume features
    'ORIGIN_FLIGHTS', 'DEST_FLIGHTS', 'ROUTE_FLIGHTS'
]

target = 'DELAYED'

# ===========================
# LOAD PARQUET (KEEP AS-IS)
# ===========================
print("\n📂 Loading parquet files from S3...")
train_df = pd.read_parquet(f's3://{bucket}/{prefix}/features/train_features.parquet')
val_df = pd.read_parquet(f's3://{bucket}/{prefix}/features/val_features.parquet')
test_df = pd.read_parquet(f's3://{bucket}/{prefix}/features/test_features.parquet')

print(f"✓ Train: {len(train_df):,} rows × {len(train_df.columns)} columns")
print(f"✓ Val:   {len(val_df):,} rows × {len(val_df.columns)} columns")
print(f"✓ Test:  {len(test_df):,} rows × {len(test_df.columns)} columns")

# ===========================
# SELECT ONLY MODEL FEATURES
# ===========================
print(f"\n🔧 Selecting {len(model_features)} numeric features + target...")

# Target first, then features (XGBoost expects this)
columns_for_csv = [target] + model_features

train_csv = train_df[columns_for_csv]
val_csv = val_df[columns_for_csv]
test_csv = test_df[columns_for_csv]

print(f"✓ CSV shape: {train_csv.shape[1]} columns (1 target + {len(model_features)} features)")

# Verify all numeric
print(f"\n✅ Data types check:")
non_numeric = train_csv.select_dtypes(exclude=['int64', 'float64']).columns
if len(non_numeric) > 0:
    print(f"   ❌ WARNING: Non-numeric columns found: {non_numeric.tolist()}")
else:
    print(f"   ✅ All columns are numeric!")

# ===========================
# SAVE AS CSV (NO HEADER, NO INDEX)
# ===========================
print(f"\n💾 Saving as CSV...")
train_csv.to_csv('train.csv', header=False, index=False)
val_csv.to_csv('val.csv', header=False, index=False)
test_csv.to_csv('test.csv', header=False, index=False)

print("✓ Local CSV files created")

# Show sample
print(f"\n📄 First row of train.csv (first 10 values):")
print(train_csv.iloc[0, :10].values)

# ===========================
# UPLOAD TO S3
# ===========================
print(f"\n☁️  Uploading to S3...")
s3 = boto3.client('s3')

s3.upload_file('train.csv', bucket, f'{prefix}/data/train/train.csv')
s3.upload_file('val.csv', bucket, f'{prefix}/data/validation/val.csv')
s3.upload_file('test.csv', bucket, f'{prefix}/data/test/test.csv')

print("✓ Files uploaded:")
print(f"  📁 s3://{bucket}/{prefix}/data/train/train.csv")
print(f"  📁 s3://{bucket}/{prefix}/data/validation/val.csv")
print(f"  📁 s3://{bucket}/{prefix}/data/test/test.csv")

# ===========================
# VERIFY UPLOADS
# ===========================
print(f"\n🔍 Verifying S3 uploads...")
for folder in ['train', 'validation', 'test']:
    response = s3.list_objects_v2(
        Bucket=bucket, 
        Prefix=f'{prefix}/data/{folder}/'
    )
    for obj in response.get('Contents', []):
        size_mb = obj['Size'] / (1024 * 1024)
        print(f"   ✓ {folder}: {size_mb:.1f} MB")

print("\n" + "="*70)
print("🎉 TRAINING CSVs GENERATED")
print("="*70)
print(f"\n📊 Model will train on: {len(model_features)} features")
print(f"🎯 Target: {target}")
print(f"\n📂 Original parquet files unchanged:")
print(f"   s3://{bucket}/{prefix}/features/*.parquet")
print(f"   (Contains all columns including AIRLINE, ORIGIN_AIRPORT, etc.)")
print(f"\n📄 New CSV files for training:")
print(f"   s3://{bucket}/{prefix}/data/train/train.csv")
print(f"   (Contains only 20 numeric features + target)")
print(f"\n✅ Ready to run pipeline!")
print("="*70)

GENERATING TRAINING CSVs FROM PARQUET

📂 Loading parquet files from S3...
✓ Train: 4,299,046 rows × 27 columns
✓ Val:   482,878 rows × 27 columns
✓ Test:  462,367 rows × 27 columns

🔧 Selecting 20 numeric features + target...
✓ CSV shape: 21 columns (1 target + 20 features)

✅ Data types check:
   ✅ All columns are numeric!

💾 Saving as CSV...
✓ Local CSV files created

📄 First row of train.csv (first 10 values):
[0. 1. 1. 4. 0. 5. 0. 1. 0. 0.]

☁️  Uploading to S3...
✓ Files uploaded:
  📁 s3://sagemaker-us-east-1-425709451100/aai540-group1/data/train/train.csv
  📁 s3://sagemaker-us-east-1-425709451100/aai540-group1/data/validation/val.csv
  📁 s3://sagemaker-us-east-1-425709451100/aai540-group1/data/test/test.csv

🔍 Verifying S3 uploads...
   ✓ train: 842.2 MB
   ✓ validation: 75.6 MB
   ✓ test: 91.0 MB

🎉 TRAINING CSVs GENERATED

📊 Model will train on: 20 features
🎯 Target: DELAYED

📂 Original parquet files unchanged:
   s3://sagemaker-us-east-1-425709451100/aai540-group1/features/*.p

In [5]:
import pandas as pd

# Load your parquet
train_df = pd.read_parquet('s3://sagemaker-us-east-1-425709451100/aai540-group1/features/train_features.parquet')

# Check for non-numeric columns
print("Data types:")
print(train_df.dtypes)

print("\nNon-numeric columns:")
non_numeric = train_df.select_dtypes(exclude=['int64', 'float64']).columns
print(non_numeric.tolist())

# If there are non-numeric columns, show unique values
for col in non_numeric:
    print(f"\n{col}: {train_df[col].unique()[:5]}")

Data types:
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
SCHEDULED_DEPARTURE      int64
AIRLINE                 object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
DISTANCE                 int64
SCHEDULED_TIME         float64
DELAYED                  int64
DEP_HOUR                 int64
HOUR_SIN               float64
HOUR_COS               float64
IS_PEAK_HOUR             int64
IS_WEEKEND               int64
IS_LONG_HAUL             int64
DISTANCE_BUCKET          int64
ROUTE                   object
AIRLINE_DELAY_RATE     float64
ORIGIN_DELAY_RATE      float64
DEST_DELAY_RATE        float64
ROUTE_DELAY_RATE       float64
ORIGIN_FLIGHTS         float64
DEST_FLIGHTS           float64
ROUTE_FLIGHTS          float64
flight_id               object
event_time             float64
dtype: object

Non-numeric columns:
['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ROUTE', 'flight_id']

AIRLINE: ['AS' 'AA' 'US' 'DL' 'NK']

OR

In [7]:
"""
Create inference input (features only, no target)
"""
import pandas as pd
import boto3

bucket = 'sagemaker-us-east-1-425709451100'
prefix = 'aai540-group1'

print("Creating inference input (features only)...")

# Load test data
test_df = pd.read_csv(f's3://{bucket}/{prefix}/data/test/test.csv', header=None)

print(f"Original shape: {test_df.shape}")
print(f"Columns: {test_df.shape[1]} (1 target + 20 features)")

# Remove first column (target)
test_features_only = test_df.iloc[:, 1:]  # Skip column 0

print(f"Inference input shape: {test_features_only.shape}")
print(f"Columns: {test_features_only.shape[1]} (20 features only)")

# Save locally
test_features_only.to_csv('test_inference.csv', header=False, index=False)

# Upload to S3
s3 = boto3.client('s3')
s3.upload_file('test_inference.csv', bucket, f'{prefix}/data/inference/test_inference.csv')

print(f"\n✅ Inference input created:")
print(f"   s3://{bucket}/{prefix}/data/inference/test_inference.csv")
print(f"\n✅ Ready to re-run batch transform with correct input")

Creating inference input (features only)...
Original shape: (462367, 21)
Columns: 21 (1 target + 20 features)
Inference input shape: (462367, 20)
Columns: 20 (20 features only)

✅ Inference input created:
   s3://sagemaker-us-east-1-425709451100/aai540-group1/data/inference/test_inference.csv

✅ Ready to re-run batch transform with correct input
